In [1]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [2]:
# load input features and make layer
version='v16_250104'
sub_v=''
stastic_v=f'{version}{sub_v}'
# stastic_v=f'{version}'
output_dir = os.path.join(r'J:\lakemapping\postprocess',version)

auxiliary_dataset_gdb=r'J:\lakemapping\auxiliary_dataset.gdb'
stastics_gdb=os.path.join(output_dir,f'0_summary_stastics{sub_v}.gdb')
river_mask_gdb=os.path.join(output_dir,'2_polygon_iw_River.gdb')
after_mask_gdb=os.path.join(output_dir,f'3_polygon_after_rivermask.gdb')
mwBL_gdb=os.path.join(output_dir,f'4_polygon_afm_mergeWith_BigLake.gdb')
merge_data_gdb=os.path.join(output_dir,f'4_merge_process.gdb')
merge_gdb=os.path.join(output_dir,f'5_polygon_afm_mergeWith_GLAKES_PLD.gdb')
point_gdb=os.path.join(output_dir,f'6_point{sub_v}.gdb')
excel_dir=os.path.join(output_dir,f'stastics_excel{sub_v}')
os.makedirs(excel_dir, exist_ok=True)
new_PLD_gdb=r'J:\lakemapping\PLD\PLD.gdb'
temp_gdb=os.path.join(output_dir,f'7_correct_temp_file{sub_v}.gdb')#
if not os.path.exists(mwBL_gdb):
    arcpy.CreateFileGDB_management(output_dir,'4_polygon_afm_mergeWith_BigLake.gdb')
if not os.path.exists(stastics_gdb):
    arcpy.CreateFileGDB_management(output_dir,f'0_summary_stastics{sub_v}.gdb')
if not os.path.exists(point_gdb):
    arcpy.CreateFileGDB_management(output_dir,f'6_point{sub_v}.gdb')
if not os.path.exists(merge_gdb):
    arcpy.CreateFileGDB_management(output_dir,f'5_polygon_afm_mergeWith_GLAKES_PLD{sub_v}.gdb')
if not os.path.exists(merge_data_gdb):
    arcpy.CreateFileGDB_management(output_dir,f'4_merge_process.gdb')
if not os.path.exists(temp_gdb):
    arcpy.CreateFileGDB_management(output_dir,f'7_correct_temp_file{sub_v}.gdb')

In [3]:
print(stastic_v)

v16_250104


In [4]:
PLD_Count_Area_SJ_lyr='Global_grid_shp_1degree_PLD_Count_Area'

In [5]:
seven_continents=['Asia','Africa','Europe','Oceania_Anta','North_America','South_America']
eight_continents=['Asia','Siberia','Africa','Europe','Oceania_Anta','North_America','Arctic','South_America']

In [6]:
Arid_region_lyr='Global_AI_Reclassify_Clip_simple'
PLD_list=['af','eu','si','as','au','sa','na','ar','gr']

In [7]:
osm_reservoir=os.path.join(auxiliary_dataset_gdb,'osm_natural_water_reservoir')
arcpy.MakeFeatureLayer_management(osm_reservoir,'osm_reservoir')

GeoDAR=r'D:\lakemapping\0_auxiliary_data\GeoDAR\GeoDAR_v10_v11\GeoDAR_v11_reservoirs.shp'
arcpy.MakeFeatureLayer_management(GeoDAR,'GeoDAR')

<Result 'GeoDAR'>

In [8]:
GLAKES_Res=os.path.join(auxiliary_dataset_gdb,'GLAKES_Res')
arcpy.MakeFeatureLayer_management(GLAKES_Res,'GLAKES_Res')

# GLAKES_natural_lake=os.path.join(auxiliary_dataset_gdb,'GLAKES_gte1_natural_lake')
# arcpy.MakeFeatureLayer_management(GLAKES_natural_lake,'GLAKES_natural_lake')

<Result 'GLAKES_Res'>

In [8]:
SHIFT=os.path.join(auxiliary_dataset_gdb,'SHIFT')
SHIFT_lyr='SHIFT'
# arcpy.MakeFeatureLayer_management(SHIFT,SHIFT_lyr)

In [71]:
continent=r'D:\lakemapping\0_auxiliary_data\World_Continents\World_Continents.shp'
continent_lyr='continent'
arcpy.MakeFeatureLayer_management(continent,continent_lyr)

ar=os.path.join(auxiliary_dataset_gdb,'ar')
arcpy.MakeFeatureLayer_management(ar,'ar')

ar=os.path.join(auxiliary_dataset_gdb,'si')
arcpy.MakeFeatureLayer_management(ar,'si')

<Result 'si'>

In [7]:
polygon_afm_mwPG_Count_SJ=os.path.join(temp_gdb,'Global_grid_shp_1degree_hu_Count')
polygon_afm_mwPG_Count_SJ_lyr='Global_grid_shp_1degree_hu_Count'
polygon_afm_mwPG_Count_Area_SJ=os.path.join(temp_gdb,'Global_grid_shp_1degree_hu_Count_Area')
polygon_afm_mwPG_Count_Area_SJ_lyr='Global_grid_shp_1degree_hu_Count_Area'

In [8]:
lakes_iwBR_SJ=os.path.join(river_mask_gdb,'total_iwBR_arm_SJ')
total_lakes_arm=os.path.join(river_mask_gdb,'total_lakes_arm_singlepart')
lakes_iwBR_afterMask=os.path.join(after_mask_gdb,'lakes_iwBR_afterMask')
total_lakes_arm_lyr='lakes_arm'
lakes_iwBR_SJ_lyr='lakes_iwBR_arm_SJ'
lakes_iwBR_afm_lyr='lakes_iwBR_afm'

In [9]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

def cal_continent(lyr):
    arcpy.AddField_management(lyr, 'Continent', "Text")
    for i in range(0,6):
        continent=seven_continents[i]
        selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
        selectByAttribute(lyr,'NEW_SELECTION',"Continent IS NULL")
        selectByLocation(lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
        calculateField(lyr,'Continent',f"'{continent}'")
    selectByAttribute(lyr,'NEW_SELECTION',"Continent = 'Asia'")
    selectByLocation(lyr,'HAVE_THEIR_CENTER_IN','si','SUBSET_SELECTION')
    arcpy.CalculateField_management(lyr, "continent","'Siberia'")
    selectByLocation(lyr,'HAVE_THEIR_CENTER_IN','ar','NEW_SELECTION')
    arcpy.CalculateField_management(lyr, "Continent","'Arctic'")
        
def add_Res_flag(inputFeature):
    selectByAttribute(inputFeature,'NEW_SELECTION', 'flag IS NULL OR "flag" <= 1 ')
    selectByLocation(inputFeature,'INTERSECT',osm_reservoir,'SUBSET_SELECTION')
    arcpy.CalculateField_management(inputFeature, 'flag',  3)

    selectByAttribute(inputFeature,'NEW_SELECTION', 'flag IS NULL OR "flag" <= 1 ')
    selectByLocation(inputFeature,'INTERSECT',GeoDAR,'SUBSET_SELECTION')
    arcpy.CalculateField_management(inputFeature, 'flag',  4)

In [10]:
def cal_lake_type(lyr,area_column):
    lake_list=[0,0.01,0.03,0.1,1,100,1000000]
    arcpy.AddField_management(lyr,'lake_type','Short')
    for i in range(6):
        selectByAttribute(lyr,'NEW_SELECTION',f"{area_column}>={lake_list[i]} AND {area_column}<{lake_list[i+1]}")
        arcpy.CalculateField_management(lyr,'lake_type',i)

In [11]:
GLAKES=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES.gdb\GLAKES'
GLAKES_new_gdb=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES_after_30p_land_mask.gdb'
GLAKES_iwSHIFT=os.path.join(GLAKES_new_gdb,'au1_GLAKES_iwSHIFT')
GLAKES_niwSHIFT=os.path.join(GLAKES_new_gdb,'au1_GLAKES_niwSHIFT')

GLAKES_lyr='GLAKES'
GLAKES_iwSHIFT_lyr='au1_GLAKES_iwSHIFT'
GLAKES_niwSHIFT_lyr='au1_GLAKES_niwSHIFT'
GLAKES_land_mask_lyr='au1_GLAKES_land_mask_iwSHIFT'
GLAKES_iwSHIFT_clip=os.path.join(GLAKES_new_gdb,'GLAKES_iwSHIFT_clip')
GLAKES_iwSHIFT_clip_sp=os.path.join(GLAKES_new_gdb,'GLAKES_iwSHIFT_clip_sp')
GLAKES_iwSHIFT_clip_sp_lyr='GLAKES_iwSHIFT_clip_sp'
GLAKES_iwSHIFT_afm=os.path.join(GLAKES_new_gdb,'au2_GLAKES_iwSHIFT_after_mask_gt3ha')
GLAKES_iwSHIFT_afm_lyr='GLAKES_iwSHIFT_after_mask_gt3ha'
GLAKES_iwSHIFT_afm_keep_arid=os.path.join(GLAKES_new_gdb,'au2_GLAKES_iwSHIFT_after_mask_gt3ha_keep_arid')
GLAKES_iwSHIFT_afm_keep_arid_lyr='au2_GLAKES_iwSHIFT_after_mask_gt3ha_keep_arid'

In [12]:
GLAKES_gte1_Rser=os.path.join(merge_data_gdb,'GLAKES_gte1_Rser_eraser')
GLAKES_gte1_Rser_lyr='GLAKES_gte1_Rser'

GLAKES_merge_file=os.path.join(merge_data_gdb,'GLAKES_for_merge')
GLAKES_merge_file_point=os.path.join(merge_data_gdb,'GLAKES_for_merge_point')

pond_region=r'D:\lakemapping\10_paper_writting\figure\8_visualize.gdb\Region_1_pond_for_GLAKES_merge'
GLAKES_merge_file_lyr='GLAKES_for_merge'
GLAKES_merge_file_point_lyr='GLAKES_for_merge_point'

In [13]:
a_iwBL_merge_with_BL_raw=os.path.join(after_mask_gdb,'a_total_lakes_afm_iwBL_merge_with_BigLakes_raw')
a_iwBL_merge_with_BL=os.path.join(after_mask_gdb,'a_total_lakes_afm_iwBL_merge_with_BigLakes')
a_iwBL_merge_with_BL_raw_lyr='a_lakes_afm_iwBL_mw_BL_raw'
a_iwBL_merge_with_BL_lyr='a_lakes_afm_iwBL_mw_BL'

In [14]:
big_lakes=os.path.join(after_mask_gdb,f'total_big_lakes')
big_lakes_lyr=f'big_lakes'

In [15]:
three_dataset_dissolve_SJ=os.path.join(merge_data_gdb,'three_dataset_dissolve_SJ')
three_dataset_dissolve_SJ_lyr=f'three_dataset_dissolve_SJ'
three_dataset_dissolve_SJ_SJ=os.path.join(merge_data_gdb,'three_dataset_dissolve_SJ_SJ')
three_dataset_dissolve_SJ_SJ_lyr='three_dataset_dissolve_SJ_SJ'

In [16]:
PLD_Arid=os.path.join(merge_data_gdb,'PLD_Arid')
PLD_Arid_point=os.path.join(merge_data_gdb,'PLD_Arid_point')

Arid_region=r'D:\lakemapping\0_auxiliary_data\AI\Global-AI_ET0_annual_v3\Global-AI_ET0_v3_annual\Global_AI_Reclassify_Clip_simple.shp'
Arid_region_lyr='Global_AI_Reclassify_Clip_simple'

three_dataset_merge=os.path.join(merge_data_gdb,'three_dataset_merge')
three_dataset_merge_lyr=f'three_dataset_merge'
three_dataset_dissolve=os.path.join(merge_data_gdb,'three_dataset_dissolve')
three_dataset_dissolve_lyr=f'three_dataset_dissolve'

b_merge_file=os.path.join(merge_data_gdb,'b_for_merge')
b_merge_file_point=os.path.join(merge_data_gdb,'b_for_merge_point')

three_dataset_dissolve_for_merge=os.path.join(mwBL_gdb,'three_dataset_dissolve_for_merge')
b_merge_layer='three_dataset_dissolve_for_merge'

In [104]:
arcpy.MakeFeatureLayer_management(Arid_region,Arid_region_lyr)

<Result 'Global_AI_Reclassify_Clip_simple'>

## 1 处理GLAKES

In [26]:
arcpy.MakeFeatureLayer_management(GLAKES,GLAKES_lyr)

<Result 'GLAKES'>

### 1.1 选择shift内的，mask

In [142]:
# selectByLocation(GLAKES_lyr,'INTERSECT',SHIFT_lyr,'NEW_SELECTION')
# arcpy.conversion.ExportFeatures(GLAKES_lyr,GLAKES_iwSHIFT)
# selectByAttribute(GLAKES_lyr,'SWITCH_SELECTION','')
# selectByLocation(GLAKES_lyr,'INTERSECT',SHIFT_lyr,'SWITCH_SELECTION')
# arcpy.conversion.ExportFeatures(GLAKES_lyr,GLAKES_niwSHIFT)

<Result 'D:\\lakemapping\\0_auxiliary_data\\GLAKES\\GLAKES_after_30p_land_mask.gdb\\au1_GLAKES_niwSHIFT'>

In [ ]:
arcpy.MakeFeatureLayer_management(GLAKES_iwSHIFT,GLAKES_iwSHIFT_lyr)
# arcpy.AddField_management(GLAKES_iwSHIFT_lyr,'Aird_Region','Short')
# selectByLocation(GLAKES_iwSHIFT_lyr, 'INTERSECT', Arid_region_lyr,'SUBSET_SELECTION')
# calculateField(GLAKES_iwSHIFT_lyr, "Aird_Region",2)

In [ ]:
arcpy.analysis.Clip(GLAKES_iwSHIFT_lyr, GLAKES_land_mask_lyr, GLAKES_iwSHIFT_clip)
arcpy.management.MultipartToSinglepart(GLAKES_iwSHIFT_clip,GLAKES_iwSHIFT_clip_sp)
arcpy.MakeFeatureLayer_management(GLAKES_iwSHIFT_clip_sp,GLAKES_iwSHIFT_clip_sp_lyr)

In [18]:
# arcpy.AddField_management(GLAKES_iwSHIFT_clip_sp_lyr,'Area_bound','Double')
# arcpy.CalculateField_management(GLAKES_iwSHIFT_clip_sp_lyr, "Area_bound",  "!shape.geodesicArea@SQUAREKILOMETERS!")
# selectByAttribute(GLAKES_iwSHIFT_clip_sp_lyr,'NEW_SELECTION',f"Area_bound>0.03")
# arcpy.conversion.ExportFeatures(GLAKES_iwSHIFT_clip_sp_lyr,GLAKES_iwSHIFT_afm)
arcpy.MakeFeatureLayer_management(GLAKES_iwSHIFT_afm,GLAKES_iwSHIFT_afm_lyr)

<Result 'GLAKES_iwSHIFT_after_mask_gt3ha'>

In [149]:
# selectByAttribute(GLAKES_iwSHIFT_lyr,'NEW_SELECTION','Aird_Region=2')
# selectByAttribute(GLAKES_iwSHIFT_after_mask_gt3ha_lyr,'NEW_SELECTION','Aird_Region IS NULL')
# arcpy.Merge_management([GLAKES_iwSHIFT_lyr,GLAKES_iwSHIFT_afm_lyr],GLAKES_iwSHIFT_afm_keep_arid)
arcpy.MakeFeatureLayer_management(GLAKES_iwSHIFT_afm_keep_arid,GLAKES_iwSHIFT_afm_keep_arid_lyr)

<Result 'au2_GLAKES_iwSHIFT_after_mask_gt3ha_keep_arid'>

In [151]:
arcpy.MakeFeatureLayer_management(GLAKES_niwSHIFT,GLAKES_niwSHIFT_lyr)

<Result 'au1_GLAKES_niwSHIFT'>

In [152]:
selectByAttribute(GLAKES_niwSHIFT_lyr,'NEW_SELECTION','"Area_bound" >= 1 OR "Rser_flag"=1')
selectByAttribute(GLAKES_iwSHIFT_afm_keep_arid_lyr,'NEW_SELECTION','"Area_bound" >= 1 OR "Rser_flag"=1')
arcpy.Merge_management([GLAKES_niwSHIFT_lyr,GLAKES_iwSHIFT_afm_keep_arid_lyr],GLAKES_gte1_Rser)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\4_merge_process.gdb\\GLAKES_gte1_Rser'>

In [154]:
# arcpy.MakeFeatureLayer_management(GLAKES_gte1_Rser,GLAKES_gte1_Rser_lyr)
# selectByLocation(GLAKES_gte1_Rser_lyr, 'INTERSECT',pond_region)
# arcpy.AddField_management(GLAKES_gte1_Rser_lyr,'operate','Short')
# calculateField(GLAKES_gte1_Rser_lyr,'operate',-1)
selectByLocation(GLAKES_gte1_Rser_lyr, 'ARE_IDENTICAL_TO','GLAKES_gte1_Rs_ExportFeature1')
calculateField(GLAKES_gte1_Rser_lyr,'operate',-2)

In [156]:
selectByAttribute('GLAKES_gte1_Rser_op_2_after_mask','NEW_SELECTION','operate =1')
selectByAttribute(GLAKES_gte1_Rser_lyr,'NEW_SELECTION','operate IS NULL')
arcpy.Merge_management(['GLAKES_gte1_Rser_op_2_after_mask',GLAKES_gte1_Rser_lyr],GLAKES_merge_file)
arcpy.management.FeatureToPoint(GLAKES_merge_file,GLAKES_merge_file_point,'INSIDE')
arcpy.MakeFeatureLayer_management(GLAKES_merge_file_point,GLAKES_merge_file_point_lyr)

<Result 'GLAKES_for_merge_point'>

In [ ]:
for i in range(0,8):
    continent=eight_continents[i]
    print(continent)
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','area IS NULL')
#     out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_lakes_merge_with_BL_stastics')
#     arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['lake_area',"sum"]], ['mwGLAKES_PLD']) 

Asia
Siberia
Africa
Europe
Oceania_Anta
North_America
Arctic
South_America


In [ ]:
input_table_b_list=[]
for i in range(0,8):
    continent=eight_continents[i]
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_lakes_merge_with_BL_stastics')
    input_table_b_list.append(out_table)
file_name=f'{stastic_v}_b_sum_lakes_afm_mwBL_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v15_241128\\stastics_excel\\v15_241128_sum_lakes_afm_mwBL_stastics.xls'>

## 2. 获得river mask后结果，与Big Lake合并

In [24]:
arcpy.MakeFeatureLayer_management(lakes_iwBR_SJ,lakes_iwBR_SJ_lyr)
# arcpy.MakeFeatureLayer_management(total_lakes_arm,total_lakes_arm_lyr)

<Result 'lakes_iwBR_arm_SJ'>

In [25]:
out_table=os.path.join(stastics_gdb,'iwBR_stastics')
selectByAttribute(lakes_iwBR_SJ_lyr,'NEW_SELECTION',"")
arcpy.analysis.Statistics(lakes_iwBR_SJ_lyr, out_table, [['area',"sum"]], ['Continent','operate']) 
file_name='iwBR_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(out_table, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\stastics_excel\\iwBR_stastics.xls'>

In [117]:
selectByAttribute(lakes_iwBR_SJ_lyr,'NEW_SELECTION','"operate" = 0.5')
selectByLocation(total_lakes_arm_lyr, 'WITHIN', lakes_iwBR_SJ_lyr,'NEW_SELECTION')
selectByAttribute(total_lakes_arm_lyr,'SUBSET_SELECTION','"operate" IS NULL')
selectByAttribute(total_lakes_arm_lyr,'ADD_TO_SELECTION','"operate" = 1')
selectByAttribute(lakes_iwBR_SJ_lyr,'NEW_SELECTION','"operate">= 1')
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(lakes_iwBR_SJ_lyr)#未mask
fieldmappings_all.addTable(total_lakes_arm_lyr)
field_list = ['area','flag','Continent','iwBL']
for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
arcpy.Merge_management([lakes_iwBR_SJ_lyr,total_lakes_arm_lyr],lakes_iwBR_afterMask,fieldmappings)
arcpy.MakeFeatureLayer_management(lakes_iwBR_afterMask,lakes_iwBR_afm_lyr)
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION','area IS NULL')
arcpy.CalculateField_management(lakes_iwBR_afm_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")

<Result 'lakes_iwBR_afm'>

In [27]:
arcpy.MakeFeatureLayer_management(lakes_iwBR_afterMask,lakes_iwBR_afm_lyr)
out_table=os.path.join(stastics_gdb,'a_iwBR_afm_stastics')
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION',"")
arcpy.analysis.Statistics(lakes_iwBR_afm_lyr, out_table, [['area',"sum"]], ['Continent','flag']) 
file_name='a_iwBR_afm_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(out_table, Output_Excel_File)

ExecuteError: Failed to execute. Parameters are not valid.
WARNING 000725: Output Table: Dataset J:\lakemapping\postprocess\v16_250104\0_summary_stastics.gdb\a_iwBR_afm_stastics already exists.
ERROR 000728: Field operate does not exist within table
Failed to execute (Statistics).


In [32]:
arcpy.MakeFeatureLayer_management(big_lakes,big_lakes_lyr)
selectByLocation(lakes_iwBR_afm_lyr, 'INTERSECT', big_lakes_lyr,'NEW_SELECTION')
# arcpy.AddField_management(lakes_iwBR_afm_lyr, 'iwBL', "short")
calculateField(lakes_iwBR_afm_lyr, 'iwBL',1)
selectByAttribute(lakes_iwBR_afm_lyr,'SWITCH_SELECTION',"")
calculateField(lakes_iwBR_afm_lyr, 'iwBL',0)

In [34]:
out_table=os.path.join(stastics_gdb,'a_iwBR_afm_column_iwBL_stastics')
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION',"")
arcpy.analysis.Statistics(lakes_iwBR_afm_lyr, out_table, [['area',"sum"]], ['Continent','iwBL']) 
file_name='a_iwBR_afm_column_iwBL_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(out_table, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\stastics_excel\\a_iwBR_afm_column_iwBL_stastics.xls'>

In [29]:
input_table_afm_list=[]
for i in range(0,8):
    continent=eight_continents[i]
    lakes_niwBR_afm_lyr='lakes_'+continent+'_niwBR_afm'
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_{continent}_niwBR_afm_stastics')
    arcpy.analysis.Statistics(lakes_niwBR_afm_lyr, out_table, [['area',"sum"]], ['iwBL']) 
    input_table_afm_list.append(out_table)
file_name=f'a_{stastic_v}_niwBR_afm_iwBL_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_afm_list, Output_Excel_File)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Table: Dataset lakes_Asia_niwBR_afm does not exist or is not supported
WARNING 000725: Output Table: Dataset J:\lakemapping\postprocess\v16_250104\0_summary_stastics.gdb\v16_250104_Asia_niwBR_afm_stastics already exists.
Failed to execute (Statistics).


In [121]:
merge_list=[lakes_iwBR_afm_lyr]
selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION',"iwBL = 1")
for continent in eight_continents:
    lakes_niwBR_afm_lyr='lakes_'+continent+'_niwBR_afm'
    lakes_niwBR_afm=os.path.join(after_mask_gdb,'lakes_'+continent+'_niwBR_afterMask')
    arcpy.MakeFeatureLayer_management(lakes_niwBR_afm,lakes_niwBR_afm_lyr)
    merge_list.append(lakes_niwBR_afm_lyr)
    selectByAttribute(lakes_niwBR_afm_lyr,'NEW_SELECTION',"iwBL = 1")
#把iwBL=1的湖泊与big lake文件合并，并融合
merge_list.append(big_lakes_lyr)
print(merge_list)

start=time.time()
arcpy.Merge_management(merge_list,a_iwBL_merge_with_BL_raw)
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL_raw, a_iwBL_merge_with_BL_raw_lyr)#global_lakes_afm_merged_with_GBL
end=time.time()
print(f'time:{end-start}s')

['lakes_iwBR_afm', 'lakes_Asia_niwBR_afm', 'lakes_Siberia_niwBR_afm', 'lakes_Africa_niwBR_afm', 'lakes_Europe_niwBR_afm', 'lakes_Oceania_Anta_niwBR_afm', 'lakes_North_America_niwBR_afm', 'lakes_Arctic_niwBR_afm', 'lakes_South_America_niwBR_afm', 'big_lakes']
time:47.76441311836243s


In [122]:
start=time.time()
print('start Dissolve')
arcpy.Dissolve_management(a_iwBL_merge_with_BL_raw_lyr, a_iwBL_merge_with_BL,"", "", "SINGLE_PART")#dissolve copylake
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr,"area","Double")
arcpy.CalculateField_management(a_iwBL_merge_with_BL_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
end=time.time()
print(f'time:{end-start}s')

start Dissolve
time:1868.932942390442s


In [123]:
arcpy.management.DeleteIdentical(a_iwBL_merge_with_BL_lyr,['area','Shape_Length','Shape_Area'],"5 Meters")

<Result 'a_lakes_afm_iwBL_mw_BL'>

In [125]:
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'iwBL', "short")
calculateField(a_iwBL_merge_with_BL_lyr, "iwBL",1)
cal_continent(a_iwBL_merge_with_BL_lyr)

In [127]:
# arcpy.management.Delete(a_iwBL_merge_with_BL_raw)
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'flag', "short")
# selectByLocation(a_iwBL_merge_with_BL_lyr, 'INTERSECT', GLAKES_Res)
calculateField(a_iwBL_merge_with_BL_lyr, "flag",2)
add_Res_flag(a_iwBL_merge_with_BL_lyr)

In [35]:
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)
out_table=os.path.join(stastics_gdb,f'a_iwBL_merge_with_BL_stastics')
selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',"")
arcpy.analysis.Statistics(a_iwBL_merge_with_BL_lyr, out_table, [['area',"sum"]], ['continent']) 
file_name=f'a_iwBL_merge_with_BL_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(out_table, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\stastics_excel\\a_iwBL_merge_with_BL_stastics.xls'>

In [ ]:
arcpy.MakeFeatureLayer_management(GLAKES_merge_file,GLAKES_merge_file_lyr)
arcpy.MakeFeatureLayer_management(PLD_Arid,'PLD_Arid')

In [41]:
for i in range(0,8):
    continent=eight_continents[i]
    print(continent)
#     selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',f"Continent = '{continent}'")

#     selectByAttribute(lakes_iwBR_afm_lyr,'NEW_SELECTION',f"iwBL = 0 AND Continent = '{continent}'")
#     lakes_niwBR_afm=os.path.join(after_mask_gdb,'lakes_'+continent+'_niwBR_afterMask')
#     lakes_niwBR_afm_lyr='lakes_'+continent+'_niwBR_afm'
# #     arcpy.MakeFeatureLayer_management(lakes_niwBR_afm,lakes_niwBR_afm_lyr)
    
#     selectByAttribute(lakes_niwBR_afm_lyr,'NEW_SELECTION',f"iwBL = 0")
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_polygon_afm_mwBL'
    
#     field_list=['area','iwBL','flag']
#     fieldmappings = arcpy.FieldMappings()
#     fieldmappings_all = arcpy.FieldMappings()
#     fieldmappings_all.addTable(lakes_niwBR_afm_lyr)
#     for field_i in field_list:
#         field_idx = fieldmappings_all.findFieldMapIndex(field_i)
#         fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
        
#     arcpy.Merge_management([lakes_niwBR_afm_lyr,a_iwBL_merge_with_BL_lyr,lakes_iwBR_afm_lyr],b_lake_afm_merge_with_BL,field_mappings=fieldmappings)
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
#     arcpy.AddField_management(b_lake_afm_merge_with_BL_lyr, 'lake_area', "DOUBLE")
#     arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr, 'lake_area','!area!')
#     cal_lake_type(b_lake_afm_merge_with_BL_lyr,'lake_area')
    
#     arcpy.AddField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD','Short')
#     selectByLocation(b_lake_afm_merge_with_BL_lyr, 'INTERSECT',GLAKES_merge_file_lyr,'NEW_SELECTION')
#     arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD',1)
#     selectByAttribute(b_lake_afm_merge_with_BL_lyr,'SWITCH_SELECTION',"")
#     selectByLocation(b_lake_afm_merge_with_BL_lyr, 'INTERSECT', 'PLD_Arid','SUBSET_SELECTION')
#     arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD',2)
    
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','')
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_lakes_merge_with_BL_stastics')
    arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['flag']) 

Asia
Siberia
Africa
Europe
Oceania_Anta
North_America
Arctic
South_America


In [56]:
# arcpy.Merge_management([lakes_niwBR_afm_lyr,a_iwBL_merge_with_BL_lyr,lakes_iwBR_afm_lyr],b_lake_afm_merge_with_BL,field_mappings=fieldmappings)
arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','')
out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_lakes_merge_with_BL_stastics')
arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['flag']) 

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\0_summary_stastics.gdb\\v16_250104_b6_North_America_lakes_merge_with_BL_stastics'>

In [19]:
input_table_b_list=[]
for i in range(0,8):
    continent=eight_continents[i]
    print(continent)
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_polygon_afm_mwBL'
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','area IS NULL')
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_mwGLAKES_PLD_stastics')
#     arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['mwGLAKES_PLD'])
#     input_table_b_list.append(out_table)
# file_name=f'{stastic_v}_b_sum_mwGLAKES_PLD_stasticss.xls'
# Output_Excel_File=os.path.join(excel_dir,file_name)
# arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

Asia
Siberia
Africa
Europe
Oceania_Anta
North_America
Arctic
South_America


In [42]:
input_table_b_list=[]
for i in range(0,8):
    continent=eight_continents[i]
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_lakes_merge_with_BL_stastics')
    input_table_b_list.append(out_table)
file_name=f'{stastic_v}_sum_b_lakes_afm_mwBL_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\stastics_excel\\v16_250104_sum_lakes_afm_mwBL_stastics.xls'>

## 3 和PLD,GLAKES融合

In [26]:
# merge_list=[]
# for region in PLD_list:#[1,3,4,5]:
#     print(region)
#     PLD_region=os.path.join(new_PLD_gdb,f'PLD_{region}')
#     PLD_region_lyr=f'PLD_{region}'
#     arcpy.MakeFeatureLayer_management(PLD_region,PLD_region_lyr)
# #     arcpy.AddField_management(PLD_region_lyr,'iwSHIFT','Short')
# #     selectByLocation(PLD_region_lyr,'INTERSECT','SHIFT')
# #     arcpy.CalculateField_management(PLD_region_lyr,'iwSHIFT',1)
# #     arcpy.AddField_management(PLD_region_lyr,'Arid_flag','Short')
# #     selectByLocation(PLD_region_lyr,'INTERSECT','Global_AI_Reclassify_Clip_simple')
# #     arcpy.CalculateField_management(PLD_region_lyr,'Arid_flag',1)
#     selectByAttribute(PLD_region_lyr,'NEW_SELECTION','Arid_flag=1')
#     merge_list.append(PLD_region_lyr)

af
eu
si
as
au
sa
na
ar
gr


In [18]:
merge_list=[]
gdb=r'D:\lakemapping\10_paper_writting\figure\8_visualize.gdb'
sample_result=os.path.join(gdb,'Data_5_PLD')
lyr='Region_5_for_visualize'
for region in PLD_list:#[1,3,4,5]:
    print(region)
    PLD_region=os.path.join(new_PLD_gdb,f'PLD_{region}')
    PLD_region_lyr=f'PLD_{region}'
    arcpy.MakeFeatureLayer_management(PLD_region,PLD_region_lyr)
    selectByLocation(PLD_region_lyr, 'INTERSECT',lyr,'NEW_SELECTION')
    merge_list.append(PLD_region_lyr)
arcpy.Merge_management(merge_list,sample_result)

af
eu
si
as
au
sa
na
ar
gr


<Result 'D:\\lakemapping\\10_paper_writting\\figure\\8_visualize.gdb\\Data_5_PLD'>

In [52]:
# arcpy.Merge_management(merge_list,PLD_Arid)
# arcpy.management.FeatureToPoint(PLD_Arid,PLD_Arid_point,'INSIDE')
PLD_Arid_point_lyr='PLD_Arid_point'
PLD_Arid_lyr='PLD_Arid'
arcpy.MakeFeatureLayer_management(PLD_Arid,PLD_Arid_lyr)
arcpy.MakeFeatureLayer_management(PLD_Arid_point,PLD_Arid_point_lyr)

<Result 'PLD_Arid_point'>

In [161]:
merge_list=[]

for i in range(0,8):
    continent=eight_continents[i]
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
# #     arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
# #     arcpy.management.DeleteField(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD')
    arcpy.AddField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD','Short')
    selectByLocation(b_lake_afm_merge_with_BL_lyr, 'INTERSECT',GLAKES_merge_file_lyr,'NEW_SELECTION')
    arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD',1)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'SWITCH_SELECTION',"")
    selectByLocation(b_lake_afm_merge_with_BL_lyr, 'INTERSECT', 'PLD_Arid','SUBSET_SELECTION')
    arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD',2)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','mwGLAKES_PLD>0')
    merge_list.append(b_lake_afm_merge_with_BL_lyr)
arcpy.Merge_management(merge_list,b_merge_file)
arcpy.management.FeatureToPoint(b_merge_file,b_merge_file_point,'INSIDE')

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\4_merge_process.gdb\\b_for_merge_point'>

In [21]:
input_table_b_list=[]
for i in range(0,8):
    continent=eight_continents[i]
    print(continent)
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
#     arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
#     selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','area IS NULL')
    out_table=os.path.join(stastics_gdb,f'{stastic_v}_b{i+1}_{continent}_mwGLAKES_PLD_stastics')
#     arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['mwGLAKES_PLD']) 
    input_table_b_list.append(out_table)
file_name=f'b_{stastic_v}_mwGLAKES_PLD_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

Asia
Siberia
Africa
Europe
Oceania_Anta
North_America
Arctic
South_America


<Result 'J:\\lakemapping\\postprocess\\v16_250104\\stastics_excel\\b_v16_250104_mwGLAKES_PLD_stastics.xls'>

In [68]:
# print(b_lake_afm_merge_with_BL_lyr)
# # arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD',1)
# # selectByAttribute(b_lake_afm_merge_with_BL_lyr,'SWITCH_SELECTION',"")
# # selectByLocation(b_lake_afm_merge_with_BL_lyr, 'INTERSECT', 'PLD_Arid','SUBSET_SELECTION')
# arcpy.CalculateField_management(b_lake_afm_merge_with_BL_lyr,'mwGLAKES_PLD',2)
# selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','mwGLAKES_PLD>0')
# merge_list.append(b_lake_afm_merge_with_BL_lyr)

b8_South_America_lake_afm_merge_with_BL


In [58]:
arcpy.MakeFeatureLayer_management(b_merge_file,'b_for_merge')

<Result 'b_for_merge'>

In [163]:
arcpy.Merge_management(['b_for_merge',GLAKES_merge_file_lyr,'PLD_Arid'],three_dataset_merge)
arcpy.MakeFeatureLayer_management(three_dataset_merge,three_dataset_merge_lyr)
arcpy.Dissolve_management(three_dataset_merge_lyr, three_dataset_dissolve,"", "", "SINGLE_PART")

arcpy.MakeFeatureLayer_management(three_dataset_dissolve,three_dataset_dissolve_lyr)
arcpy.AddField_management(three_dataset_dissolve_lyr,"lake_area","Double")
arcpy.CalculateField_management(three_dataset_dissolve_lyr, "lake_area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
arcpy.AddField_management(three_dataset_dissolve_lyr,"continent","Text")

In [57]:
arcpy.MakeFeatureLayer_management(three_dataset_dissolve,three_dataset_dissolve_lyr)
# arcpy.MakeFeatureLayer_management(b_merge_file_point,'b_for_merge_point')

# fieldmappings = arcpy.FieldMappings()
# fieldmappings_all = arcpy.FieldMappings()
# fieldmappings_all.addTable(three_dataset_dissolve_lyr)
# fieldmappings_all.addTable('b_for_merge_point')
# field_list=['lake_area','area']#
# for field_i in field_list:
#     field_idx = fieldmappings_all.findFieldMapIndex(field_i)
#     field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
#     fieldmappings.addFieldMap(field_fieldmap)
# area_idx = fieldmappings.findFieldMapIndex("area")
# area_fieldmap = fieldmappings.getFieldMap(area_idx)
# area_fieldmap.mergeRule = "sum"
# fieldmappings.replaceFieldMap(area_idx, area_fieldmap)

# print('Conduct spatial join')
# arcpy.SpatialJoin_analysis(three_dataset_dissolve_lyr, 'b_for_merge_point', three_dataset_dissolve_SJ, "JOIN_ONE_TO_ONE",
#                            "KEEP_ALL", fieldmappings, "CONTAINS")
# arcpy.MakeFeatureLayer_management(three_dataset_dissolve_SJ,three_dataset_dissolve_SJ_lyr)

<Result 'three_dataset_dissolve'>

In [50]:
arcpy.MakeFeatureLayer_management(GLAKES_merge_file_point,GLAKES_merge_file_point_lyr)
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(three_dataset_dissolve_SJ_lyr)
fieldmappings_all.addTable(GLAKES_merge_file_point_lyr)
field_list=['lake_area','area','Join_Count','Area_bound']#
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    if field_i=='area':
        outputField=field_fieldmap.outputField
        outputField.name='sum_area_b'
        outputField.alias='sum_area_b'
        field_fieldmap.outputField=outputField
    if field_i=='Join_Count':
        outputField=field_fieldmap.outputField
        outputField.name='count_b'
        outputField.alias='count_b'
        field_fieldmap.outputField=outputField
    fieldmappings.addFieldMap(field_fieldmap)
    
area_idx = fieldmappings.findFieldMapIndex("Area_bound")
area_fieldmap = fieldmappings.getFieldMap(area_idx)
area_fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(area_idx, area_fieldmap)

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(three_dataset_dissolve_SJ, GLAKES_merge_file_point_lyr, three_dataset_dissolve_SJ_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")
arcpy.MakeFeatureLayer_management(three_dataset_dissolve_SJ_SJ,three_dataset_dissolve_SJ_SJ_lyr)

Conduct spatial join


<Result 'three_dataset_dissolve_SJ_SJ'>

In [53]:
# arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr,"Hu","short")
# selectByLocation(three_dataset_dissolve_SJ_SJ_lyr,'contains','b_for_merge_point')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"Hu",1)

# arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr,"GLAKES","short")
# selectByLocation(three_dataset_dissolve_SJ_SJ_lyr,'contains',GLAKES_merge_file_point_lyr)
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"GLAKES",1)

# arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr,"PLD","short")
selectByLocation(three_dataset_dissolve_SJ_SJ_lyr,'contains',PLD_Arid_point_lyr)
arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"PLD",1)

<Result 'three_dataset_dissolve_SJ_SJ'>

In [54]:
arcpy.MakeFeatureLayer_management(Arid_region,Arid_region_lyr)

<Result 'Global_AI_Reclassify_Clip_simple'>

In [55]:
arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr, 'change_ratio', "DOUBLE")#新建字段为面积比例
selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','sum_area_b IS NOT NULL')
arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'change_ratio', '(!lake_area!-!sum_area_b!)/!sum_area_b!*100')
arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate', "short")
arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr, 'Arid_flag', "short")
selectByLocation(three_dataset_dissolve_SJ_SJ_lyr,'INTERSECT',Arid_region_lyr)
arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'Arid_flag',2)

<Result 'three_dataset_dissolve_SJ_SJ'>

In [71]:
# arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr, 'Oceanline', "short")
selectByLocation(three_dataset_dissolve_SJ_SJ_lyr,'INTERSECT','lines')
arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'Oceanline',2)
selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Oceanline IS NULL')
arcpy.SelectLayerByLocation_management(three_dataset_dissolve_SJ_SJ_lyr,'INTERSECT','lines',search_distance='10 kilometers',selection_type='SUBSET_SELECTION')
arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'Oceanline',1)

<Result 'three_dataset_dissolve_SJ_SJ'>

In [68]:
# arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate', "short")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','change_ratio=0 or lake_area<0.005')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate',0)
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','operate IS NULL AND change_ratio < 5 AND PLD=1 AND GLAKES IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate',0)
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','operate IS NULL AND change_ratio < 50 And lake_area < 1 AND PLD=1 AND GLAKES IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate',0)
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','count_b = 1 AND operate IS NULL And change_ratio < 5 And lake_area < 1')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate',0)
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Arid_flag = 2 AND operate IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr, 'operate',1)
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','count_b = 1 AND operate IS NULL And change_ratio < 5')
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','count_b>2 AND operate IS NULL AND Oceanline >0 AND Rser_flag IS NULL AND max_area_b <1')
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','operate IS NULL And change_ratio < 5 AND PLD=1 AND GLAKES IS NULL')
selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION',"operate IS NULL And Hu IS NULL AND PLD IS NULL AND GLAKES =1")


In [72]:
cal_continent(three_dataset_dissolve_SJ_SJ_lyr)

In [82]:
# arcpy.AddField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","Text")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','GLAKES =1 and Hu IS NULL and PLD IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'GLAKES'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','PLD =1 and GLAKES IS NULL and Hu IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'PLD'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Hu =1 and GLAKES =1 and PLD IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'Hu,GLAKES'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Hu =1 and GLAKES IS NULL and PLD =1')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'Hu,PLD'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Hu is null and GLAKES =1 and PLD =1')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'GLAKES,PLD'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Hu =1 and GLAKES =1 and PLD =1')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'Hu,GLAKES,PLD'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','Hu =1 and GLAKES IS NULL and PLD IS NULL')
# arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'Hu,GLAKES'")
# selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','sources IS NULL')
arcpy.CalculateField_management(three_dataset_dissolve_SJ_SJ_lyr,"sources","'GLAKES'")

<Result 'three_dataset_dissolve_SJ_SJ'>

In [83]:
selectByAttribute(three_dataset_dissolve_SJ_SJ_lyr,'NEW_SELECTION','operate IS NULL OR operate>0')
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(three_dataset_dissolve_SJ_SJ_lyr)
field_list=['lake_area','sources','Continent']#,'Arid_flag'
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    fieldmappings.addFieldMap(field_fieldmap)
arcpy.conversion.ExportFeatures(three_dataset_dissolve_SJ_SJ_lyr,three_dataset_dissolve_for_merge,field_mapping=fieldmappings)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\4_polygon_afm_mergeWith_BigLake.gdb\\three_dataset_dissolve_for_merge'>

In [24]:
arcpy.MakeFeatureLayer_management(three_dataset_dissolve_for_merge,b_merge_layer)
# arcpy.MakeFeatureLayer_management(b_merge_file_point,'b_for_merge_point')

<Result 'three_dataset_dissolve_for_merge'>

In [85]:
cal_lake_type(b_merge_layer,'lake_area')

ExecuteError: ERROR 160706: Cannot acquire a lock.
Failed to execute (CalculateField).


### 生成c

In [29]:
arcpy.SelectLayerByLocation_management('b_for_merge_point', 'WITHIN',b_merge_layer,selection_type='NEW_SELECTION',invert_spatial_relationship=True)
for i in range(6,7):
    continent=eight_continents[i]
    print(continent)
    b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    c_lake_afm_mw_GP=os.path.join(merge_gdb,f'c{i+1}_{continent}_lakes')
    c_lake_afm_mw_GP_lyr=f'c{i+1}_{continent}_lakes'

    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION',"mwGLAKES_PLD IS NULL")
    selectByLocation(b_lake_afm_merge_with_BL_lyr, 'CONTAINS', 'b_for_merge_point','ADD_TO_SELECTION')
    selectByAttribute(b_merge_layer,'NEW_SELECTION',f"Continent = '{continent}'")
    field_list=['lake_area','lake_type','sources']#,'Arid_flag'
    fieldmappings = arcpy.FieldMappings()
    fieldmappings_all = arcpy.FieldMappings()
    fieldmappings_all.addTable(b_merge_layer)
    fieldmappings_all.addTable(b_lake_afm_merge_with_BL_lyr)
    for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))

    arcpy.Merge_management([b_lake_afm_merge_with_BL_lyr,b_merge_layer],c_lake_afm_mw_GP,field_mappings=fieldmappings)
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP,c_lake_afm_mw_GP_lyr)
#     selectByAttribute(c_lake_afm_mw_GP_lyr,'NEW_SELECTION',"")
#     out_table=os.path.join(stastics_gdb,f'{version}_c{i+1}_{continent}_lakes_stastics')
#     arcpy.analysis.Statistics(c_lake_afm_mw_GP_lyr, out_table, [['lake_area',"sum"]], ['sources']) 

Arctic


In [203]:
input_table_b_list=[]
for i in range(0,8):
    continent=eight_continents[i]
#     print(continent)
#     c_lake_afm_mw_GP=os.path.join(merge_gdb,f'c{i+1}_{continent}_lakes')
#     c_lake_afm_mw_GP_lyr=f'c{i+1}_{continent}_lakes'
#     arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP,c_lake_afm_mw_GP_lyr)
#     selectByAttribute(c_lake_afm_mw_GP_lyr,'NEW_SELECTION','lake_area IS NULL')
    out_table=os.path.join(stastics_gdb,f'{version}_c{i+1}_{continent}_lakes_stastics')
    arcpy.analysis.Statistics(c_lake_afm_mw_GP_lyr, out_table, [['lake_area',"sum"]], ['sources']) 
file_name=f'{stastic_v}_c_sum_lakes_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

In [ ]:
arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP,c_lake_afm_mw_GP_lyr)
#     selectByAttribute(c_lake_afm_mw_GP_lyr,'NEW_SELECTION',"")
out_table=os.path.join(stastics_gdb,f'{version}_c{i+1}_{continent}_lakes_stastics')
arcpy.analysis.Statistics(c_lake_afm_mw_GP_lyr, out_table, [['lake_area',"sum"]], ['sources']) 

<Result 'J:\\lakemapping\\postprocess\\v15_241128\\0_summary_stastics.gdb\\v15_241128_c6_North_America_lakes_stastics'>

In [23]:
input_table_b_list=[]
for i in range(0,8):
    continent=eight_continents[i]
    out_table=os.path.join(stastics_gdb,f'{version}_c{i+1}_{continent}_lakes_stastics')
    input_table_b_list.append(out_table)
file_name=f'{stastic_v}_c_sum_lakes_stastics.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\stastics_excel\\v16_250104_c_sum_lakes_stastics.xls'>

## 空间分布分析

### 计算lake type 生成pi

In [39]:
grid=os.path.join(temp_gdb,'Global_grid_shp_1degree')
grid_lyr='Global_grid_shp_1degree'
# arcpy.MakeFeatureLayer_management(grid,grid_lyr)

In [55]:
point_merge_list=[]
pi_point_merge_list=[]
for i in [5]:#range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_GP=os.path.join(merge_gdb,f'c{i+1}_{continent}_lakes')
    c_lake_afm_mw_GP_lyr=f'c{i+1}_{continent}_lakes'
    c_lake_afm_mw_GP_point=os.path.join(point_gdb,f'c{i+1}_{continent}_lakes_point')
    c_lake_afm_mw_GP_point_lyr=f'c{i+1}_{continent}_lakes_point'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP,c_lake_afm_mw_GP_lyr)
    
    c_lake_afm_mw_GP_pi=os.path.join(temp_gdb,f'c{i+1}_{continent}_lakes_pi')
    c_lake_afm_mw_GP_pi_lyr=f'c{i+1}_{continent}_lakes_pi'
    c_lake_afm_mw_GP_pi_point=os.path.join(point_gdb,f'c{i+1}_{continent}_lakes_pi_point')
    c_lake_afm_mw_GP_pi_point_lyr=f'c{i+1}_{continent}_lakes_pi_point'
#     cal_lake_type(c_lake_afm_mw_GP_lyr,'lake_area')
    
    print('start transform to point')
#     selectByAttribute(c_lake_afm_mw_GP_lyr,'NEW_SELECTION','') 
    arcpy.management.FeatureToPoint(c_lake_afm_mw_GP_lyr,c_lake_afm_mw_GP_point,'INSIDE')
    print('start pairwise intersect')
#     arcpy.analysis.PairwiseIntersect([c_lake_afm_mw_GP_lyr, grid_lyr],c_lake_afm_mw_GP_pi,join_attributes='NO_FID')
#     arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP_pi,c_lake_afm_mw_GP_pi_lyr)
#     selectByLocation(c_lake_afm_mw_GP_pi_lyr,'SHARE_A_LINE_SEGMENT_WITH',grid_lyr,'NEW_SELECTION')
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_pi_lyr,'lake_area', "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
#     selectByAttribute(c_lake_afm_mw_GP_pi_lyr,'NEW_SELECTION','') 
#     arcpy.management.FeatureToPoint(c_lake_afm_mw_GP_pi_lyr,c_lake_afm_mw_GP_pi_point,'INSIDE')


North_America
start transform to point
start pairwise intersect


In [26]:
arcpy.analysis.PairwiseIntersect([c_lake_afm_mw_GP_lyr, grid_lyr],c_lake_afm_mw_GP_pi,join_attributes='NO_FID')
arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP_pi,c_lake_afm_mw_GP_pi_lyr)
selectByLocation(c_lake_afm_mw_GP_pi_lyr,'SHARE_A_LINE_SEGMENT_WITH',grid_lyr,'NEW_SELECTION')
arcpy.CalculateField_management(c_lake_afm_mw_GP_pi_lyr,'lake_area', "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
selectByAttribute(c_lake_afm_mw_GP_pi_lyr,'NEW_SELECTION','') 
arcpy.management.FeatureToPoint(c_lake_afm_mw_GP_pi_lyr,c_lake_afm_mw_GP_pi_point,'INSIDE')

<Result 'J:\\lakemapping\\postprocess\\v16_250104\\6_point.gdb\\c1_Asia_lakes_pi_point'>

In [34]:
total_lake_afm_mwPG_point=os.path.join(point_gdb,'total_lakes_point')
total_lake_afm_mwPG_pi_point=os.path.join(point_gdb,'total_lakes_point_pi_new')
total_lake_afm_mwPG_point_lyr='total_polygon_afm_mwPG_point'
total_lake_afm_mwPG_pi_point_lyr='total_polygon_afm_mwPG_pi_point'

In [35]:
merge_list=[]
for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_GP_point=os.path.join(point_gdb,f'c{i+1}_{continent}_lakes_point')
    c_lake_afm_mw_GP_point_lyr=f'c{i+1}_{continent}_lakes_point'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP_point,c_lake_afm_mw_GP_point_lyr)
#     arcpy.AddField_management(c_lake_afm_mw_GP_point_lyr,'hu_c1','Short')
#     arcpy.AddField_management(c_lake_afm_mw_GP_point_lyr,'hu_c3','Short')
#     arcpy.AddField_management(c_lake_afm_mw_GP_point_lyr,'hu_c10','Short')
#     arcpy.AddField_management(c_lake_afm_mw_GP_point_lyr,'hu_c100','Short')
#     selectByAttribute(c_lake_afm_mw_GP_point_lyr,'NEW_SELECTION',"lake_type=0")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_point_lyr,'hu_c1',1)
#     selectByAttribute(c_lake_afm_mw_GP_point_lyr,'NEW_SELECTION',"lake_type=1")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_point_lyr,'hu_c3',1)
#     selectByAttribute(c_lake_afm_mw_GP_point_lyr,'NEW_SELECTION',"lake_type=2")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_point_lyr,'hu_c10',1)
#     selectByAttribute(c_lake_afm_mw_GP_point_lyr,'NEW_SELECTION',"lake_type=3")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_point_lyr,'hu_c100',1)
#     selectByAttribute(c_lake_afm_mw_GP_point_lyr,'NEW_SELECTION','')
    merge_list.append(c_lake_afm_mw_GP_point_lyr)
arcpy.Merge_management(merge_list,total_lake_afm_mwPG_point)

Asia
Siberia
Africa
Europe
Oceania_Anta
North_America
Arctic
South_America


<Result 'J:\\lakemapping\\postprocess\\v16_250104\\6_point.gdb\\total_lakes_point'>

In [37]:
merge_list=[]
for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    print(continent)
    c_lake_afm_mw_GP_pi_point=os.path.join(point_gdb,f'c{i+1}_{continent}_lakes_pi_point')
    c_lake_afm_mw_GP_pi_point_lyr=f'c{i+1}_{continent}_lakes_pi_point'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP_pi_point,c_lake_afm_mw_GP_pi_point_lyr)
#     arcpy.AddField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a1','Double')
#     arcpy.AddField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a3','Double')
#     arcpy.AddField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a10','Double')
#     arcpy.AddField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a100','Double')
#     selectByAttribute(c_lake_afm_mw_GP_pi_point_lyr,'NEW_SELECTION',"lake_type=0")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a1','!lake_area!')
#     selectByAttribute(c_lake_afm_mw_GP_pi_point_lyr,'NEW_SELECTION',"lake_type=1")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a3','!lake_area!')
#     selectByAttribute(c_lake_afm_mw_GP_pi_point_lyr,'NEW_SELECTION',"lake_type=2")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a10','!lake_area!')
#     selectByAttribute(c_lake_afm_mw_GP_pi_point_lyr,'NEW_SELECTION',"lake_type=3")
#     arcpy.CalculateField_management(c_lake_afm_mw_GP_pi_point_lyr,'hu_a100','!lake_area!')
#     selectByAttribute(c_lake_afm_mw_GP_pi_point_lyr,'NEW_SELECTION','')
    merge_list.append(c_lake_afm_mw_GP_pi_point_lyr)
arcpy.Merge_management(merge_list,total_lake_afm_mwPG_pi_point)

Asia
Siberia
Africa
Europe
Oceania_Anta
North_America
Arctic
South_America


<Result 'J:\\lakemapping\\postprocess\\v16_250104\\6_point.gdb\\total_lakes_point_pi_new'>

In [20]:
# arcpy.AddField_management(total_lake_afm_mwPG_point_lyr,'hu_c1','Short')
# arcpy.AddField_management(total_lake_afm_mwPG_point_lyr,'hu_c3','Short')
# arcpy.AddField_management(total_lake_afm_mwPG_point_lyr,'hu_c10','Short')
# arcpy.AddField_management(total_lake_afm_mwPG_point_lyr,'hu_c100','Short')
# selectByAttribute(total_lake_afm_mwPG_point_lyr,'NEW_SELECTION',"lake_type=0")
# arcpy.CalculateField_management(total_lake_afm_mwPG_point_lyr,'hu_c1',1)
# selectByAttribute(total_lake_afm_mwPG_point_lyr,'NEW_SELECTION',"lake_type=1")
# arcpy.CalculateField_management(total_lake_afm_mwPG_point_lyr,'hu_c3',1)
# selectByAttribute(total_lake_afm_mwPG_point_lyr,'NEW_SELECTION',"lake_type=2")
# arcpy.CalculateField_management(total_lake_afm_mwPG_point_lyr,'hu_c10',1)
# selectByAttribute(total_lake_afm_mwPG_point_lyr,'NEW_SELECTION',"lake_type=3")
# arcpy.CalculateField_management(total_lake_afm_mwPG_point_lyr,'hu_c100',1)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Table: Dataset total_polygon_afm_mwPG_point does not exist or is not supported
Failed to execute (AddField).


In [ ]:
# arcpy.AddField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a1','Double')
# arcpy.AddField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a3','Double')
# arcpy.AddField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a10','Double')
# arcpy.AddField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a100','Double')
# selectByAttribute(total_lake_afm_mwPG_pi_point_lyr,'NEW_SELECTION',"lake_type=0")
# arcpy.CalculateField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a1','!lake_area!')
# selectByAttribute(total_lake_afm_mwPG_pi_point_lyr,'NEW_SELECTION',"lake_type=1")
# arcpy.CalculateField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a3','!lake_area!')
# selectByAttribute(total_lake_afm_mwPG_pi_point_lyr,'NEW_SELECTION',"lake_type=2")
# arcpy.CalculateField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a10','!lake_area!')
# selectByAttribute(total_lake_afm_mwPG_pi_point_lyr,'NEW_SELECTION',"lake_type=3")
# arcpy.CalculateField_management(total_lake_afm_mwPG_pi_point_lyr,'hu_a100','!lake_area!')

<Result 'total_polygon_afm_mwPG_pi_point'>

In [23]:
merge_list=[]
gdb=r'D:\lakemapping\10_paper_writting\figure\8_visualize.gdb'
sample_result=os.path.join(gdb,'Data_5_GLAKES_plus')
lyr='Region_5_for_visualize'
for i in range(0,8):#[1,3,4,5]:
    continent=eight_continents[i]
    c_lake_afm_mw_GP=os.path.join(merge_gdb,f'c{i+1}_{continent}_lakes')
    c_lake_afm_mw_GP_lyr=f'c{i+1}_{continent}_lakes'
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_GP,c_lake_afm_mw_GP_lyr)
    selectByLocation(c_lake_afm_mw_GP_lyr, 'INTERSECT',lyr,'NEW_SELECTION')
    merge_list.append(c_lake_afm_mw_GP_lyr)
arcpy.Merge_management(merge_list,sample_result)

<Result 'D:\\lakemapping\\10_paper_writting\\figure\\8_visualize.gdb\\Data_5_GLAKES_plus'>

In [41]:
arcpy.MakeFeatureLayer_management(total_lake_afm_mwPG_point,total_lake_afm_mwPG_point_lyr)
arcpy.MakeFeatureLayer_management(total_lake_afm_mwPG_pi_point,total_lake_afm_mwPG_pi_point_lyr)

<Result 'total_polygon_afm_mwPG_pi_point'>

In [42]:
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(grid_lyr)
field_list = ['grid_area','Lat_down','Lon_left','Grid_ID']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
    
fieldmappings_all.addTable(total_lake_afm_mwPG_point_lyr)
field_list = ['hu_c1','hu_c3','hu_c10','hu_c100']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    field_fieldmap.mergeRule = "sum"
    fieldmappings.addFieldMap(field_fieldmap)

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(grid_lyr, total_lake_afm_mwPG_point_lyr, polygon_afm_mwPG_Count_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")
arcpy.MakeFeatureLayer_management(polygon_afm_mwPG_Count_SJ, polygon_afm_mwPG_Count_SJ_lyr)

fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(polygon_afm_mwPG_Count_SJ_lyr)
field_list = ['grid_area','Join_Count','hu_c1','hu_c3','hu_c10','hu_c100','Lat_down','Lon_left','Grid_ID']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    if field_i=='Join_Count':
        outputField=field_fieldmap.outputField
        outputField.name='hu_c0'
        outputField.alias='hu_c0'
        field_fieldmap.outputField=outputField
    fieldmappings.addFieldMap(field_fieldmap)

fieldmappings_all.addTable(total_lake_afm_mwPG_pi_point_lyr)
field_list = ['lake_area','hu_a1','hu_a3','hu_a10','hu_a100']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    field_fieldmap.mergeRule = "sum"
    if field_i=='lake_area':
        outputField=field_fieldmap.outputField
        outputField.name='hu_a0'
        outputField.alias='hu_a0'
        field_fieldmap.outputField=outputField
    fieldmappings.addFieldMap(field_fieldmap)

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(polygon_afm_mwPG_Count_SJ_lyr, total_lake_afm_mwPG_pi_point_lyr, polygon_afm_mwPG_Count_Area_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")
arcpy.MakeFeatureLayer_management(polygon_afm_mwPG_Count_Area_SJ, polygon_afm_mwPG_Count_Area_SJ_lyr)

Conduct spatial join
Conduct spatial join


<Result 'Global_grid_shp_1degree_hu_Count_Area'>

In [38]:
arcpy.MakeFeatureLayer_management(polygon_afm_mwPG_Count_Area_SJ, polygon_afm_mwPG_Count_Area_SJ_lyr)

<Result 'Global_grid_shp_1degree_hu_Count_Area'>

In [43]:
field_list=['hu_c0','hu_c1','hu_c3','hu_c10','hu_c100','hu_a0','hu_a1','hu_a3','hu_a10','hu_a100']
for field in field_list:
    selectByAttribute(polygon_afm_mwPG_Count_Area_SJ_lyr,'NEW_SELECTION',f'{field} IS NULL')
    arcpy.CalculateField_management(polygon_afm_mwPG_Count_Area_SJ_lyr,field,0)
selectByAttribute(polygon_afm_mwPG_Count_Area_SJ_lyr,'NEW_SELECTION','')

In [45]:
# target_lyr=polygon_afm_mwPG_Count_Area_SJ_lyr#polygon_afm_mwBL_BG_Count_Area_SJ_lyr
# join_lyr=PLD_Count_Area_SJ_lyr

# arcpy.AddField_management(target_lyr,'hu_ap','double')
# arcpy.AddField_management(target_lyr,'hu_ap_small_lake','double')
# calculateField(target_lyr,'hu_ap','!hu_a0!/!grid_area!*100')
# selectByAttribute(target_lyr,"NEW_SELECTION",'"hu_a0"<> 0')
# calculateField(target_lyr,'hu_ap_small_lake','(!hu_a1!+!hu_a3!+!hu_a10!+!hu_a100!)/!hu_a0!*100')

# arcpy.AddField_management(target_lyr,'diff_c0','Long')
# arcpy.AddField_management(target_lyr,'diff_c_gt1','Long')
# arcpy.AddField_management(target_lyr,'diff_c3','Long')
# arcpy.AddField_management(target_lyr,'diff_c10','Long')
# arcpy.AddField_management(target_lyr,'diff_c100','Long')

# arcpy.AddField_management(target_lyr,'diff_a0','double')
# arcpy.AddField_management(target_lyr,'diff_a3','double')
# arcpy.AddField_management(target_lyr,'diff_a10','double')
# arcpy.AddField_management(target_lyr,'diff_a100','double')

arcpy.management.AddJoin(target_lyr,'Grid_ID',join_lyr,'Grid_ID','keep_all')

calculateField(target_lyr,'diff_c0',f'!{target_lyr}.hu_c0!-!{join_lyr}.pld_c0!')
calculateField(target_lyr,'diff_c_gt1',f'!{target_lyr}.diff_c0!-!{target_lyr}.hu_c1!')
calculateField(target_lyr,'diff_c3',f'!{target_lyr}.hu_c3!-!{join_lyr}.pld_c3!')
calculateField(target_lyr,'diff_c10',f'!{target_lyr}.hu_c10!-!{join_lyr}.pld_c10!')
calculateField(target_lyr,'diff_c100',f'!{target_lyr}.hu_c100!-!{join_lyr}.pld_c100!')
calculateField(target_lyr,'diff_a0',f'!{target_lyr}.hu_a0!-!{join_lyr}.pld_a0!')
calculateField(target_lyr,'diff_a3',f'!{target_lyr}.hu_a3!-!{join_lyr}.pld_a3!')
calculateField(target_lyr,'diff_a10',f'!{target_lyr}.hu_a10!-!{join_lyr}.pld_a10!')
calculateField(target_lyr,'diff_a100',f'!{target_lyr}.hu_a100!-!{join_lyr}.pld_a100!')
# calculateField(target_lyr,'diff_c0','!hu_c0!-!pld_c0!')
# calculateField(target_lyr,'diff_c_gt1','!diff_c0!-!hu_c1!')
# calculateField(target_lyr,'diff_c3','!hu_c3!-!pld_c3!')
# calculateField(target_lyr,'diff_c10','!hu_c10!-!pld_c10!')
# calculateField(target_lyr,'diff_c100','!hu_c100!-!pld_c100!')
# calculateField(target_lyr,'diff_a0','!hu_a0!-!pld_a0!')
# calculateField(target_lyr,'diff_a3','!hu_a3!-!pld_a3!')
# calculateField(target_lyr,'diff_a10','!hu_a10!-!pld_a10!')
# calculateField(target_lyr,'diff_a100','!hu_a100!-!pld_a100!')

## 处理GLAKES

In [19]:
GLAKES_Count_Area_SJ=os.path.join(temp_gdb,'Global_grid_shp_1degree_GLAKES_Count_Area')
GLAKES_Count_Area_SJ_lyr='GLAKES_Count_Area_SJ'
# arcpy.MakeFeatureLayer_management(GLAKES_Count_Area_SJ, GLAKES_Count_Area_SJ_lyr)

In [28]:
for layer in [GLAKES_niwSHIFT_lyr]:#[1,3,4,5]:,GLAKES_iwSHIFT_afm_keep_arid_lyr
    print(layer)
    GLAKES_point=os.path.join(point_gdb,f'{layer}_point')
    GLAKES_point_lyr=f'{layer}_point'
    GLAKES_pi=os.path.join(temp_gdb,f'{layer}_pi')
    GLAKES_pi_lyr=f'{layer}_pi'
    GLAKES_pi_point=os.path.join(point_gdb,f'{layer}_pi_point')
    GLAKES_pi_point_lyr=f'{layer}_pi_point'
#     arcpy.AddField_management(layer,'lake_type','Short')
#     selectByAttribute(layer,'NEW_SELECTION',"Area_bound <0.1")
#     arcpy.CalculateField_management(layer,'lake_type',2)
#     selectByAttribute(layer,'NEW_SELECTION',"Area_bound>=0.1 and Area_bound <1")
#     arcpy.CalculateField_management(layer,'lake_type',3)
#     selectByAttribute(layer,'NEW_SELECTION',"Area_bound>=1 and Area_bound <100")
#     arcpy.CalculateField_management(layer,'lake_type',4)
#     selectByAttribute(layer,'NEW_SELECTION',"Area_bound>=100")
#     arcpy.CalculateField_management(layer,'lake_type',5)
    
    print('start transform to point')
#     selectByAttribute(layer,'NEW_SELECTION','') 
    arcpy.management.FeatureToPoint(layer,GLAKES_point,'INSIDE')
    print('start pairwise intersect')
    arcpy.analysis.PairwiseIntersect([layer, grid_lyr],GLAKES_pi,join_attributes='NO_FID')
    arcpy.MakeFeatureLayer_management(GLAKES_pi,GLAKES_pi_lyr)
    selectByLocation(GLAKES_pi_lyr,'SHARE_A_LINE_SEGMENT_WITH',grid_lyr,'NEW_SELECTION')
    arcpy.CalculateField_management(GLAKES_pi_lyr,'Area_bound', "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
    selectByAttribute(GLAKES_pi_lyr,'NEW_SELECTION','') 
    arcpy.management.FeatureToPoint(GLAKES_pi_lyr,GLAKES_pi_point,'INSIDE')

au1_GLAKES_niwSHIFT
start transform to point
start pairwise intersect


In [29]:
GLAKES_new_point_lyr='GLAKES_new_point'
GLAKES_new_pi_point_lyr='GLAKES_new_pi_point'

In [34]:
# arcpy.AddField_management(GLAKES_new_point_lyr,'GL_c10','Short')
# arcpy.AddField_management(GLAKES_new_point_lyr,'GL_c100','Short')
# selectByAttribute(GLAKES_new_point_lyr,'NEW_SELECTION',"lake_type=2")
# arcpy.CalculateField_management(GLAKES_new_point_lyr,'GL_c10',1)
# selectByAttribute(GLAKES_new_point_lyr,'NEW_SELECTION',"lake_type=3")
# arcpy.CalculateField_management(GLAKES_new_point_lyr,'GL_c100',1)
selectByAttribute(GLAKES_new_point_lyr,'NEW_SELECTION',"")

arcpy.AddField_management(GLAKES_new_pi_point_lyr,'GL_a10','Double')
arcpy.AddField_management(GLAKES_new_pi_point_lyr,'GL_a100','Double')
selectByAttribute(GLAKES_new_pi_point_lyr,'NEW_SELECTION',"lake_type=2")
arcpy.CalculateField_management(GLAKES_new_pi_point_lyr,'GL_a10','!Area_bound!')
selectByAttribute(GLAKES_new_pi_point_lyr,'NEW_SELECTION',"lake_type=3")
arcpy.CalculateField_management(GLAKES_new_pi_point_lyr,'GL_a100','!Area_bound!')
selectByAttribute(GLAKES_new_pi_point_lyr,'NEW_SELECTION',"")

<Result 'GLAKES_new_pi_point'>

In [32]:
GLAKES_Count_SJ=os.path.join(temp_gdb,'Global_grid_shp_1degree_GLAKES_Count')
GLAKES_Count_SJ_lyr='GLAKES_Count_SJ'
GLAKES_Count_Area_SJ=os.path.join(temp_gdb,'Global_grid_shp_1degree_GLAKES_Count_Area')
GLAKES_Count_Area_SJ_lyr='GLAKES_Count_Area_SJ'

In [91]:
calculateField(GLAKES_Count_Area_SJ_lyr,'GL_ap','!GL_a0!/!grid_area!*100')

In [35]:
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(grid_lyr)
field_list = ['grid_area','Lat_down','Lon_left','Grid_ID']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
    
fieldmappings_all.addTable(GLAKES_new_point_lyr)
field_list = ['GL_c10','GL_c100']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    field_fieldmap.mergeRule = "sum"
    fieldmappings.addFieldMap(field_fieldmap)

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(grid_lyr, GLAKES_new_point_lyr, GLAKES_Count_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")
arcpy.MakeFeatureLayer_management(GLAKES_Count_SJ, GLAKES_Count_SJ_lyr)

fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(GLAKES_Count_SJ)
field_list = ['grid_area','Join_Count','GL_c10','GL_c100','Lat_down','Lon_left','Grid_ID']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    if field_i=='Join_Count':
        outputField=field_fieldmap.outputField
        outputField.name='GL_c0'
        outputField.alias='GL_c0'
        field_fieldmap.outputField=outputField
    fieldmappings.addFieldMap(field_fieldmap)

fieldmappings_all.addTable(GLAKES_new_pi_point_lyr)
field_list = ['Area_bound','GL_a10','GL_a100']
for field_i in field_list:
    field_idx = fieldmappings_all.findFieldMapIndex(field_i)
    field_fieldmap = fieldmappings_all.getFieldMap(field_idx)
    field_fieldmap.mergeRule = "sum"
    if field_i=='Area_bound':
        outputField=field_fieldmap.outputField
        outputField.name='GL_a0'
        outputField.alias='GL_a0'
        field_fieldmap.outputField=outputField
    fieldmappings.addFieldMap(field_fieldmap)

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(GLAKES_Count_SJ_lyr, GLAKES_new_pi_point_lyr, GLAKES_Count_Area_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")
arcpy.MakeFeatureLayer_management(GLAKES_Count_Area_SJ, GLAKES_Count_Area_SJ_lyr)

Conduct spatial join
Conduct spatial join


<Result 'GLAKES_Count_Area_SJ'>

In [36]:
field_list=['GL_c0','GL_c10','GL_c100','GL_a0','GL_a10','GL_a100']
for field in field_list:
    selectByAttribute(GLAKES_Count_Area_SJ_lyr,'NEW_SELECTION',f'{field} IS NULL')
    arcpy.CalculateField_management(GLAKES_Count_Area_SJ_lyr,field,0)
selectByAttribute(GLAKES_Count_Area_SJ_lyr,'NEW_SELECTION','')

In [37]:
target_lyr=polygon_afm_mwPG_Count_Area_SJ_lyr#polygon_afm_mwBL_BG_Count_Area_SJ_lyr
join_lyr=GLAKES_Count_Area_SJ_lyr

arcpy.AddField_management(target_lyr,'diff_G_c0','Long')
arcpy.AddField_management(target_lyr,'diff_G_c_gt3','Long')
arcpy.AddField_management(target_lyr,'diff_G_c10','Long')
arcpy.AddField_management(target_lyr,'diff_G_c100','Long')

arcpy.AddField_management(target_lyr,'diff_G_a0','double')
arcpy.AddField_management(target_lyr,'diff_G_a10','double')
arcpy.AddField_management(target_lyr,'diff_G_a100','double')

arcpy.management.AddJoin(target_lyr,'Grid_ID',join_lyr,'Grid_ID','keep_all')

calculateField(target_lyr,'diff_G_c0',f'!{target_lyr}.hu_c0!-!{join_lyr}.GL_c0!')
calculateField(target_lyr,'diff_G_c_gt3',f'!{target_lyr}.diff_c0!-!{target_lyr}.GL_c3!')
calculateField(target_lyr,'diff_G_c10',f'!{target_lyr}.hu_c10!-!{join_lyr}.GL_c10!')
calculateField(target_lyr,'diff_c100',f'!{target_lyr}.hu_c100!-!{join_lyr}.GL_c100!')
calculateField(target_lyr,'diff_a0',f'!{target_lyr}.hu_a0!-!{join_lyr}.GL_a0!')
calculateField(target_lyr,'diff_a10',f'!{target_lyr}.hu_a10!-!{join_lyr}.GL_a10!')
calculateField(target_lyr,'diff_a100',f'!{target_lyr}.hu_a100!-!{join_lyr}.GL_a100!')
# calculateField(target_lyr,'diff_G_c0','!hu_c0!-!GL_c0!')
# calculateField(target_lyr,'diff_G_c_gt3','!diff_c0!-!GL_c1!')
# calculateField(target_lyr,'diff_G_c3','!hu_c3!-!GL_c3!')
# calculateField(target_lyr,'diff_G_c10','!hu_c10!-!GL_c10!')
# calculateField(target_lyr,'diff_G_c100','!hu_c100!-!GL_c100!')
# calculateField(target_lyr,'diff_G_a0','!hu_a0!-!GL_a0!')
# calculateField(target_lyr,'diff_G_a10','!hu_a10!-!GL_a10!')
# calculateField(target_lyr,'diff_G_a100','!hu_a100!-!GL_a100!')

ExecuteError: ERROR 000539: Invalid field GLAKES_Count_Area_SJ.GL_c0
Failed to execute (CalculateField).


In [ ]:
## 处理HydroLAKES

In [ ]:
HydroLAKES=os.path.join(auxiliary_dataset_gdb,'Hydrolakes')
HydroLAKES_lyr='HydroLAKES'
arcpy.MakeFeatureLayer_management(HydroLAKES,HydroLAKES_lyr)
HydroLAKES_point=os.path.join(point_gdb,f'HydroLAKES_point')
HydroLAKES_point_lyr=f'HydroLAKES_point'
HydroLAKES_pi=os.path.join(temp_gdb,f'HydroLAKES_pi')
HydroLAKES_pi_lyr=f'HydroLAKES_pi'
HydroLAKES_pi_point=os.path.join(point_gdb,f'HydroLAKES_pi_point')
HydroLAKES_pi_point_lyr=f'HydroLAKES_pi_point'
arcpy.AddField_management(HydroLAKES_lyr,'lake_type','Short')
selectByAttribute(HydroLAKES_lyr,'NEW_SELECTION',"Lake_area <1")
arcpy.CalculateField_management(HydroLAKES_lyr,'lake_type',3)
selectByAttribute(HydroLAKES_lyr,'NEW_SELECTION',"Lake_area>=1 and Lake_area <100")
arcpy.CalculateField_management(HydroLAKES_lyr,'lake_type',4)
selectByAttribute(HydroLAKES_lyr,'NEW_SELECTION',"Lake_area>=100")
arcpy.CalculateField_management(HydroLAKES_lyr,'lake_type',5)
print('start transform to point')
selectByAttribute(HydroLAKES_lyr,'NEW_SELECTION','') 
arcpy.management.FeatureToPoint(HydroLAKES_lyr,HydroLAKES_point,'INSIDE')
print('start pairwise intersect')
arcpy.analysis.PairwiseIntersect([HydroLAKES_lyr, grid_lyr],HydroLAKES_pi,join_attributes='NO_FID')
arcpy.MakeFeatureLayer_management(HydroLAKES_pi,HydroLAKES_pi_lyr)
selectByLocation(HydroLAKES_pi_lyr,'SHARE_A_LINE_SEGMENT_WITH',grid_lyr,'NEW_SELECTION')
arcpy.CalculateField_management(HydroLAKES_pi_lyr,'Lake_area', "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
selectByAttribute(HydroLAKES_pi_lyr,'NEW_SELECTION','') 
arcpy.management.FeatureToPoint(HydroLAKES_pi_lyr,HydroLAKES_pi_point,'INSIDE')